In [2]:
from ecmwf.opendata import Client
import numpy as np
import pandas as pd
import geopandas as gp
import requests
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature
from datetime import datetime, timedelta
from matplotlib import pyplot
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
import matplotlib.ticker as ticker
import shapely.speedups
from shapely import Polygon
import subprocess
from dataclasses import dataclass
import xarray as xr
import cfgrib
import zipfile
import os
import shutil
import rasterio
from rasterio.transform import from_origin
from rasterio.transform import from_bounds
from rasterio.crs import CRS
#import h3
from pyproj import Transformer
import json
import pprint
#import dask
import eccodes
import pygrib
import psycopg2
from osgeo import gdal
from collections import defaultdict
from scipy.spatial import KDTree
from itertools import chain

@dataclass
class DateTimeParts:
    year: int
    month: int
    day: int
    hour: int
    month_str: str
    day_str: str
    hour_str: str
    date_str: str

    @classmethod
    def from_datetime(cls, dt: datetime):
        month_str=str(dt.month).zfill(2)
        day_str = str(dt.day).zfill(2)
        hour_str=str(dt.hour).zfill(2)
        date_str = str(dt.year)+month_str+day_str
        return cls(year=dt.year, month=dt.month, day=dt.day, hour=dt.hour, month_str=month_str, day_str=day_str, hour_str=hour_str, date_str=date_str)
    
def windCalc(u,v):
        #print('windCalc Function')
        wind_abs = np.sqrt(u**2 + v**2)
        wind_dir_trig_to = np.arctan2(u/wind_abs, v/wind_abs)
        wind_dir_trig_to_degrees = wind_dir_trig_to * 180/np.pi ## -111.6 degrees
        wind_dir = wind_dir_trig_to_degrees + 180
        return wind_abs * 2.23694 #TO MPH
def K_to_F(temp):
    temp = ((temp - 273.15) * (9/5)) + 32
    return temp

def F_to_K(temp):
    temp = ((temp - 32) * 5/9) + 273.15
    return temp

In [3]:
df_turbines = pd.read_csv('uswtdb_v7_1_20240814.csv')
print(df_turbines)
print(df_turbines.columns)

       case_id    faa_ors           faa_asn  usgs_pr_id   eia_id t_state  \
0      3123319  40-097765  2021-WTW-7879-OE         NaN  65511.0      OK   
1      3123340  40-097520  2021-WTW-7901-OE         NaN  65511.0      OK   
2      3123283  40-098169  2021-WTW-8718-OE         NaN  65511.0      OK   
3      3123452  40-097545  2021-WTW-7884-OE         NaN  65511.0      OK   
4      3123519  40-097550  2021-WTW-7881-OE         NaN  65511.0      OK   
...        ...        ...               ...         ...      ...     ...   
74828  3113467  40-088127  2021-WTW-2406-OE         NaN      NaN      OK   
74829  3053232  36-120351  2015-WTE-5788-OE         NaN      NaN      NY   
74830  3040944  48-024978   2011-WTW-352-OE     41364.0      NaN      TX   
74831  3055917  08-072245  2015-WTW-9994-OE         NaN      NaN      CO   
74832  3055918  08-072237  2015-WTW-9995-OE         NaN      NaN      CO   

             t_county   t_fips                  p_name  p_year  ...  t_ttlh  \
0       

In [12]:
df_windfarms = df_turbines[['t_fips', 'p_name', 't_cap', 'xlong', 'ylat']]
df_windfarms = df_windfarms.dropna()
print(df_windfarms)
geometry = gp.points_from_xy(x=df_windfarms['xlong'], y=df_windfarms['ylat'])
gdf_windfarms = gp.GeoDataFrame(df_windfarms, geometry=geometry)
print(gdf_windfarms)


        t_fips                         p_name   t_cap       xlong       ylat
0      40045.0                  25 Mile Creek  4200.0  -99.740372  36.489838
1      40045.0                  25 Mile Creek  4200.0  -99.733086  36.451591
2      40045.0                  25 Mile Creek  4200.0  -99.762627  36.515823
3      40045.0                  25 Mile Creek  4200.0  -99.707520  36.490849
4      40045.0                  25 Mile Creek  4200.0  -99.724426  36.491375
...        ...                            ...     ...         ...        ...
73135   2188.0  unknown Kotzebue Wind Project    65.0 -162.557327  66.834328
73627   6065.0       unknown Riverside County  3450.0 -116.557755  33.891106
73629   6065.0       unknown Riverside County  3600.0 -116.552635  33.891678
73632   6065.0       unknown Riverside County  3600.0 -116.552597  33.890198
73687  39143.0        unknown Sandusky County   100.0  -83.134529  41.440750

[71438 rows x 5 columns]
        t_fips                         p_name   t_

In [18]:
grouped_gdf = gdf_windfarms.groupby('p_name').agg({
    't_fips': 'first', # Choose how to handle 'id' (e.g., first, list, etc.)
    't_cap': 'sum',  # get total capacity
    'ylat': 'mean',  # Optionally aggregate lat/lon
    'xlong': 'mean',
    'geometry': lambda x: x.unary_union.centroid  # Compute the centroid of grouped geometries
}).reset_index()

# Ensure the result is a GeoDataFrame

#print(grouped_gdf)
gdf_windfarms_1000 = gp.GeoDataFrame(grouped_gdf, geometry=grouped_gdf['geometry'])
gdf_windfarms_1000 = (gdf_windfarms_1000[gdf_windfarms_1000['t_cap'] >= 1000]).reset_index(drop=True)

print(gdf_windfarms_1000)

/tmp/ipykernel_395504/1663232449.py:6: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  'geometry': lambda x: x.unary_union.centroid  # Compute the centroid of grouped geometries


                          p_name   t_fips     t_cap       ylat       xlong  \
0                  25 Mile Creek  40045.0  250000.0  36.466968  -99.763681   
1        30 MW Iowa DG Portfolio  19157.0   30000.0  42.006571  -93.449786   
2     6th Space Warning Squadron  25001.0    3360.0  41.753342  -70.543552   
3             AFCEE MMR Turbines  25001.0    3000.0  41.758590  -70.546550   
4                      AG Land 1  19169.0    1600.0  42.206390  -93.325691   
...                          ...      ...       ...        ...         ...   
1397               Zachary Ridge  19143.0    4000.0  43.405956  -95.656250   
1398         Zephyr Wind Project  39063.0    4500.0  41.096057  -83.641027   
1399     Zephyr Wind Project 2.0  39063.0    4500.0  41.091126  -83.637009   
1400                       Zotos  36069.0    3300.0  42.886696  -76.968052   
1401    unknown Riverside County   6065.0   10650.0  33.890994 -116.554329   

                         geometry  
0      POINT (-99.76368 36.

In [19]:
#gdf_windfarms_1000.to_csv('windfarms1000.csv', index=False)
gdf_windfarms_1000.to_file("windfarms1000.shp")

/home/jeff/modelgen/.venv/lib/python3.12/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
